In [1]:
import pandas as pd
from sklearn.model_selection import train_test_split
from sklearn.feature_extraction.text import TfidfVectorizer
from sklearn.metrics import accuracy_score
import nltk
from nltk.corpus import stopwords
import string

In [2]:
data = pd.read_csv("/kaggle/input/blaanced/final_dataset.csv", encoding="utf-8")
data.drop_duplicates(inplace=True)
data = data.dropna()
data.tail()

,Headlines,News,Date,Category
39994,প্রতারণা রুখতে ‌আমাজনের মতো স্বদেশি ই–কমার্স স...,সংবাদ প্রতিদিন ডিজিটাল ডেস্ক:‌‌ ‌দেশে হু হু কর...,1,lifestyle
39995,৫ নয়া ফিচারে আরও আকর্ষণীয় হতে চলেছে হোয়াটসঅ...,সংবাদ প্রতিদিন ডিজিটাল ডেস্ক: প্রতিযোগিতার বাজ...,1,lifestyle
39996,অতিরিক্ত ঘুমে দুঃস্বপ্ন দেখার মাত্রা বাড়ে,প্রতিদিনের ঘুমের চাহিদা মেটানো সুস্বাস্থ্যের জ...,1,lifestyle
39998,পাহাড়ের নিরিবিলি আর সবুজের কোলে রাংবাং,সংবাদ প্রতিদিন ডিজিটাল ডেস্ক: অফিস আপনাকে কটা ...,1,lifestyle
39999,আঙুল ফোটানো বা হাড়ের সংযোগস্থলে শব্দ- এটা কি ...,হাতের আঙুলে চাপ দিলেই মট করে শব্দ হয়। আমরা বলি...,2019/06/15,lifestyle


In [3]:
import re

# Load the stopwords from the text file
with open('/kaggle/input/stopwordsbn/stopwords-bn.txt', 'r', encoding='utf-8') as file:
    stopwords_list = file.read().splitlines()

# Function to remove stopwords from a text
def remove_stopwords(text):
    words = text.split()
    cleaned_words = [word for word in words if word not in stopwords_list]
    return ' '.join(cleaned_words)

# Apply the remove_stopwords function to the 'News' column
data['News'] = data['News'].astype(str).apply(remove_stopwords)

# Sample text with Bengali numerals, URLs, and punctuation marks
data['News'] = data['News'].astype(str)

# Regex pattern for Bengali numerals, URLs, and punctuation marks
bengali_numeric_pattern = r'[০১২৩৪৫৬৭৮৯]+'
url_pattern = r'https?://\S+|www\.\S+'
punctuation_pattern = r'[।,:\u0964\u0965\"\'.\(\)\-?!;|৳]+'

# Function to clean text by removing numerals, URLs, and punctuation marks
def clean_text(text):
    text = re.sub(bengali_numeric_pattern, '', text)
    text = re.sub(url_pattern, '', text)
    text = re.sub(punctuation_pattern, '', text)
    return text

# Remove Bengali numerals, URLs, and punctuation marks from the 'News' column
data['News'] = data['News'].apply(clean_text)

# Display the modified 'News' column for the first two news articles
for index, news in enumerate(data['News'].head(2), 1):
    print(f"News {index}: {news}")
    print("-" * 50)  # Separating each news article for clarity


News 1: স্টাফ রিপোর্টার কক্সবাজার  সমাবেশের তারিখ পিছিয়ে দেয়ার বিএনপির আহ্বান জানিয়েছেন  দলের সমন্বয়ক স্বাস্থ্যমন্ত্রী মোহাম্মদ নাসিম বলেন গত রাজধানীর মহানগর নাট্যমঞ্চে আওয়ামী লীগসহ  দলের সমাবেশ ঘোষণা দেয়া হয়েছে বিএনপি সংঘাত সৃষ্টির জন্যই পূর্ব নির্ধারিত সমাবেশের তারিখ দুইদিন বাড়িয়ে শনিবার করেছে কার উস্কানিতে হয়েছে খতিয়ে দরকারবিএনপি নেতৃবৃন্দের উদ্দেশে বলেন উদ্দেশ্যপ্রণোদিত হয়ে অস্থিরতা সৃষ্টি না নির্বাচনে আসুন খালি মাঠ চাই না খেলে জিততে চাই ফাউল নির্বাচনে লালকার্ড দেখাবে দেশের জনগণ ফুটবল বিশ্বকাপে নামীদামী খেলোয়াড় গোল মিস করলেও নির্বাচনে গোল মিস প্রধানমন্ত্রী শেখ হাসিনাবৃহস্পতিবার কক্সবাজারের হোটেল সিগ্যালে অনুষ্ঠিত ন্যাশনাল হোমিওপ্যাথিক কংগ্রেসে প্রধান অতিথির বক্তব্যে এসব কথা বলেন ডিএইচএমএস ডক্টরস এ্যাসোসিয়েশন কেন্দ্রীয় কমিটি কংগ্রেসের আয়োজন করে বাংলাদেশ হোমিওপ্যাথি বোর্ড কংগ্রেস আয়োজনে সহযোগিতা করে এ্যাসোসিয়েশনের সভাপতি ডাঃ মোঃ ইফতেখার উদ্দিনের সভাপতিত্বে সম্মেলনে অন্যদের রাখেন কক্সবাজার সদর আসনের সংসদ সদস্য সাইমন সারোয়ার কমল ঢাকা মহানগর দক্ষিণ আওয়ামী লীগের সম্পাদক শাহ আলম মুরাদ বাং

In [4]:
X = data['News']
y = data['Category']
X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.2, random_state=42)

In [5]:
tfidf_vectorizer = TfidfVectorizer(max_features=5000)
X_train_tfidf = tfidf_vectorizer.fit_transform(X_train)
X_test_tfidf = tfidf_vectorizer.transform(X_test)

In [6]:
from sklearn.tree import DecisionTreeClassifier

# Create a Decision Tree classifier
dt_classifier = DecisionTreeClassifier(random_state=42)

# Fit the Decision Tree model on the TF-IDF transformed data
dt_classifier.fit(X_train_tfidf, y_train)

DecisionTreeClassifier(random_state=42)

In [7]:
from sklearn.feature_extraction.text import TfidfVectorizer
import joblib

# Save the TfidfVectorizer and the transformed data using joblib
joblib.dump(tfidf_vectorizer, 'tfidf_vectorizer.joblib')

['tfidf_vectorizer.joblib']

In [8]:
y_pred = dt_classifier.predict(X_test_tfidf)

In [9]:
sample_index = 9
test_sample = X_test.iloc[sample_index]
test_sample_tfidf = tfidf_vectorizer.transform([test_sample])
predicted_category = dt_classifier.predict(test_sample_tfidf)
actual_category = y_test.iloc[sample_index]

print(f'Test Sample:\n{test_sample}')
print(f'Predicted Category: {predicted_category[0]}')
print(f'Actual Category: {actual_category}')

Test Sample:
করোনা ভাইরাস আতঙ্কে অনির্দিষ্টকালের ক্লাসপরীক্ষা বর্জনের ঘোষণা দিয়েছে ঢাকা বিশ্ববিদ্যালয়ের ঢাবি অর্থনীতি বিভাগের শিক্ষার্থীরা শনিবার  মার্চ দুপুরে আগামীকাল রবিবার  মার্চ অনির্দিষ্টকালের ক্লাসপরীক্ষা বর্জনের ঘোষণা দেয় তারাতবে ঢাবির অর্থনীতি বিভাগের চেয়ারম্যান জানিয়েছেন আগামীকাল স্বাভাবিক নিয়মেই ক্লাসপরীক্ষা চলবেঅর্থনীতি বিভাগের মাস্টার্সের শিক্ষার্থী মোমিনুল ইসলাম জানান শিক্ষার্থী একসঙ্গে ক্লাস করায় করোনা ছড়িয়ে পড়ার সম্ভাবনা রয়েছে সেজন্য ক্লাসপরীক্ষা বর্জনের সিদ্ধান্ত নিয়েছি  শিক্ষার্থীর মতামতের ভিত্তিতে সিদ্ধান্ত হয়েছেমোমিনুল জানান অনলাইন জরিপ চালিয়েছিলাম জরিপে বিভাগের শিক্ষার্থীরা উন্মুক্ত মত দিয়েছে ওপর ভিত্তি সিদ্ধান্ত নিয়েছি এছাড়া  শিক্ষার্থীর স্বাক্ষর রয়েছে স্বাক্ষরের ফাইল বিভাগের চেয়ারম্যানের নিকটে জমা দিয়েছিঅর্থনীতি বিভাগের য় বর্ষের শিক্ষার্থী নকিব বলেন সবাই মিলে সিদ্ধান্ত নিয়েছি সংকটময় মুহূর্তে বিশ্ববিদ্যালয় প্রশাসন চুপ থাকলেও থাকতে না কারণে সিনিয়রদের আলোচনা সম্মিলিতভাবে সিদ্ধান্ত নিয়েছিআরও পড়ুন  জাবিতে মুজিববর্ষ উপলক্ষে ক্রিকেট টুর্নামেন্টএ অর্থনীতি বিভাগের চেয়ারম্

In [10]:
sample_index = 90
test_sample = X_test.iloc[sample_index]
test_sample_tfidf = tfidf_vectorizer.transform([test_sample])
predicted_category = dt_classifier.predict(test_sample_tfidf)
actual_category = y_test.iloc[sample_index]

print(f'Test Sample:\n{test_sample}')
print(f'Predicted Category: {predicted_category[0]}')
print(f'Actual Category: {actual_category}')

Test Sample:
কলকাতার বুকে সন্ধ্যা নামতেই বিয়ের বাজনা বেজে উঠেছিল সৃজিতের দক্ষিণ কলকাতার ফ্ল্যাটে শনিবার  ফেব্রুয়ারি সন্ধ্যায় বসল চাঁদের হাট হয়ে বাংলাদেশি অভিনেত্রী সমাজকর্মী রাফিয়াত রশিদ মিথিলা পরিচালক সৃজিত মুখোপাধ্যায়ের জমকালো রিসেপশন পার্টিস্বাভাবিক পার্টিতে উপস্থিত টলি পাড়ার তারকারা ডিসেম্বরের  তারিখেই বিয়ে সেরেছিলেন সৃজিতমিথিলা পরিবার ঘনিষ্ঠ বন্ধুরাই বিয়েতে উপস্থিত ছিলেন রিসেপশন টলিপাড়ায় উত্তেজনা থেকেই অবশেষে তারকাদের মেলায় হয়ে রিসেপশনলাল জহরকোট কালো পাঞ্জাবিতে সেজেছিলেন সৃজিত মিথিলা পরনে লাল রঙের জামদানি শাড়ি সাজ নয় কপালে ছোট্ট টিপ কানেগলায় মানানসই গয়নারিসেপশনে উপস্থিত সৌরভ গঙ্গোপাধ্যায় প্রসেনজিৎ চট্টোপাধ্যায় গার্গী অনির্বাণ ভট্টাচার্য সৌরসেনী মাধবী মুখোপাধ্যায় অনুপম রায় রূপম ইসলাম রুদ্রুনীল ঘোষ অপর্ণা সেন রাজ চক্রবর্তী ইন্দ্রনীল বরখাসহ অনেকেবিয়ের আনুষ্ঠানিক রীতি শেষে সৃজিতমিথিলা সাংবাদিকদের মুখোমুখি হন সাংবাদিকদের সৃজিত মুখার্জি বলেন বিয়ে হয়ে গিয়েছে লোকেদের একটা গেটটুগেদার ছিল বিয়ে বিষয়টা বলেকয়ে না ভবিতব্য হয়েছে ভারতবাংলাদেশ সম্পর্কটা দর্শনের আটকে যায়নি অনুসরণ ভালো লাগছেএবার একস

In [11]:
from sklearn.metrics import accuracy_score, classification_report
print(classification_report(y_test, y_pred))

               precision    recall  f1-score   support

    economics       0.57      0.60      0.59       871
    education       0.50      0.50      0.50       978
entertainment       0.60      0.59      0.59       996
international       0.46      0.46      0.46       923
    lifestyle       0.68      0.68      0.68       818
     national       0.41      0.41      0.41       849
     politics       0.57      0.59      0.58       925
       sports       0.67      0.65      0.66       986

     accuracy                           0.56      7346
    macro avg       0.56      0.56      0.56      7346
 weighted avg       0.56      0.56      0.56      7346



In [12]:
with open('DecisionTreeFinal.txt', 'w') as file:
    for sample_index in range(100):
        test_sample = X_test.iloc[sample_index]
        test_sample_tfidf = tfidf_vectorizer.transform([test_sample])
        predicted_category = dt_classifier.predict(test_sample_tfidf)
        actual_category = y_test.iloc[sample_index]

        file.write(f'Test Sample {sample_index + 1}:\n{test_sample}\n')
        file.write(f'Predicted Category: {predicted_category[0]}\n')
        file.write(f'Actual Category: {actual_category}\n\n')


In [13]:
import joblib
joblib.dump(dt_classifier, "dt_model.joblib")

['dt_model.joblib']